In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
df_tips = sns.load_dataset("tips")
df_tips.head(3)

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3


### Aggregate: 여러 함수를 col에 동시 적용
기본구조: df.groupby(['col1'],['col2'],...).aggregate([fn1,fn2,fn3,...])

In [2]:
# 1개의 col, 함수만 적용할경우 이렇게 해도 됨
df_tips.groupby('day')[['tip']].mean()

,tip
day,
Thur,2.771452
Fri,2.734737
Sat,2.993103
Sun,3.255132


In [3]:
# 만약 2개 이상의 col과 함수를 적용할 경우 aggregate 함수가 필요
df_tips.groupby(['day','time'])[['total_bill']].aggregate([np.max,np.min,np.mean]).head(3)

total_bill                  
                  amax   amin       mean
day  time                               
Thur Lunch       43.11   7.51  17.664754
     Dinner      18.78  18.78  18.780000
Fri  Lunch       16.27   8.58  12.845714

#### Aggregate+lambda함수
df.groupby(['col1'],...).aggregate([fn1,...])에서 aggregate 대신
- .apply(lambda x: fn(x)) 형식으로 표현
- 만약 여러 개의 lambda를 사용하고 싶을 경우:

함수 선언으로 해결:
- fn1 = lambda x:x+2
- fn2 = lambda x:x**2

In [13]:
# bill_per_size
df_tips['bill_per_size'] = df_tips.apply(lambda x : x['total_bill'] / x['size'], axis=1)
df_tips

,total_bill,tip,sex,smoker,day,time,size,bill_per_size
0,16.99,1.01,Female,No,Sun,Dinner,2,8.495000
1,10.34,1.66,Male,No,Sun,Dinner,3,3.446667
2,21.01,3.50,Male,No,Sun,Dinner,3,7.003333
3,23.68,3.31,Male,No,Sun,Dinner,2,11.840000
4,24.59,3.61,Female,No,Sun,Dinner,4,6.147500
...,...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3,9.676667
240,27.18,2.00,Female,Yes,Sat,Dinner,2,13.590000
241,22.67,2.00,Male,Yes,Sat,Dinner,2,11.335000
242,17.82,1.75,Male,No,Sat,Dinner,2,8.910000


# 질문: lambda가 2개일 경우?

In [12]:
# lambda 함수 선언
fn1 = lambda x: np.mean(x.total_bill / x.size)
fn2 = lambda x: np.mean(x.tip/x.size)

In [ ]:
# lambda 2개??
df_tips['A','B'] = df_tips.groupby('day').aggregate([fn1,fn2], axis=1)

# 매핑: .replace .map

In [19]:
# 딕셔너리 형태로 데이터프레임 만들기
import pandas as pd
import numpy as np
table = {
    '일자':['2021-12-06','2021-12-07','2021-12-08','2021-12-09'],
    '가격':[1000,3000,2000,1000],
    '구매여부':['False','True','True','True'],
    '제품': ['gum','snack','beverage','gum']
}
df = pd.DataFrame(table)

In [20]:
# 만약 한두 가지 매핑이라면, replace 명령어로도 가능
df['구매여부(bool)'] = df['구매여부'].replace('True', True).replace('False', False)
df['구매여부(eng)'] = df['구매여부'].replace('True','Yes').replace('False','No')

## 딕셔너리로 매핑하기

In [21]:
# 딕셔너리로 맵핑하기: mapping할 요소가 많을 경우 필수!!
# 아래와 같이 딕셔너리 형태의 mapping table을 만들어 준다
mapping = {'gum': '껌', 'snack':'과자', 'beverage':'음료'} 
df['제품(map)'] = df['제품'].map(mapping)

# 위의 코드는 아래와 결과가 같다
# df['제품(replace)'] = df['제품'].replace('gum', '껌').replace('snack', '과자').replace('beverage','음료')
df

,일자,가격,구매여부,제품,구매여부(bool),구매여부(eng),제품(map)
0,2021-12-06,1000,False,gum,False,No,껌
1,2021-12-07,3000,True,snack,True,Yes,과자
2,2021-12-08,2000,True,beverage,True,Yes,음료
3,2021-12-09,1000,True,gum,True,Yes,껌


# df. str칼럼 분할해 새로운 칼럼 만들기

In [2]:
# total_bill의 
import pandas as pd
import numpy as np
import seaborn as sns
df_tips = sns.load_dataset("tips")
df_tips.head(3)

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3


In [4]:
# 1. vectorization(빠름): 이나 ' '같이 구분기준이 있다면 split 이용 .str.split('.').str(0)
df_tips['total_bill'] = df_tips.total_bill.astype(str) #float to str
df_tips['total_bill$'] =  df_tips.total_bill.str.split('.').str[0] # .으로 split한 후 필요한 조각 가져오기
df_tips.head(3)

,total_bill,tip,sex,smoker,day,time,size,total_bill$
0,16.99,1.01,Female,No,Sun,Dinner,2,16
1,10.34,1.66,Male,No,Sun,Dinner,3,10
2,21.01,3.50,Male,No,Sun,Dinner,3,21


In [8]:
#2. For Loop(느림): 한 행씩 칼럼에 채워넣기
for i in df_tips:
    df_tips.loc[i,'tb$'] = str(df_tips.loc[i,'total_bill']).split('.')[0]
df_tips.head(3)

KeyError: 'total_bill'

# DataFrame 만드는 방법

In [12]:
a = list(range(9))
b = np.arange(9,18).tolist()
c = np.arange(18,27).tolist()
a,b,c
pd.DataFrame({'a':a, 'b':b, 'c':c}).head(3)


,a,b,c
0,0,9,18
1,1,10,19
2,2,11,20


In [13]:
stack = []
for aa, bb, cc in zip(a, b, c):
    stack.append([aa, bb, cc])
stack
pd.DataFrame(stack).head(3)

,0,1,2
0,0,9,18
1,1,10,19
2,2,11,20


In [15]:
zip(a,b,c)